In [1]:
from pyspark import SparkConf, SparkContext

import os 
from typing import Tuple, List

In [4]:
conf = SparkConf().setMaster("local").setAppName("category-reviews-averages")
sc = SparkContext(conf=conf)

directroy = f"{os.getcwd()}/data"
filename = "restaurant_reviews.csv"

22/12/16 14:33:58 WARN Utils: Your hostname, imhaneul-ui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 172.30.1.62 instead (on interface en0)
22/12/16 14:33:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/16 14:33:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
lines = sc.textFile(f"file:///{directroy}/{filename}")
lines.collect()

['id,item,cateogry,reviews',
 '0,짜장면,중식,125',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223   ',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [8]:
header = lines.first()
filtered_header = lines.filter(lambda row: row != header)

In [9]:
filtered_header.collect()

['0,짜장면,중식,125',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223   ',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [12]:
def parse(row) -> Tuple[str, int]:
    #'0,짜장면,중식,125'
    fields: List[str] = row.split(",")
    category: str = fields[2]
    reviews: int = int(fields[3])
    
    return (category, reviews)

categoryReview = filtered_header.map(parse)

In [17]:
categoryReviewCount = categoryReview.mapValues(lambda x: (x, 1))
categoryReviewCount.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [24]:
categoryReviewCountValue = categoryReviewCount.reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))
categoryReviewCountValue.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [25]:
categoryReviewAverage = categoryReviewCountValue.mapValues(lambda x: (x[0] / x[1]))

In [26]:
categoryReviewAverage.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]